In [0]:
%sql
-- Find orders whom placed orders above the average order amount
-- using sub query

select * from orders where total_amount > 
(
  select avg(total_amount) from orders 
)
order by total_amount asc

In [0]:
%sql
-- Find customers whom placed orders above the average order amount
-- using cte

with avg_total_amount as (
  select avg(total_amount) as avg_amount  from orders
)

select * from orders where total_amount > 
(select avg_amount from avg_total_amount) 
order by total_amount asc

In [0]:
%sql
-- Find Customers whom placed orders above the average order amount
-- using join and sub query

select c.first_name, c.city, o.order_date, o.total_amount  
from customers c
inner join orders o
ON c.customer_id = o.customer_id
where o.total_amount > 
(select avg(total_amount) from orders)
order by o.total_amount asc

In [0]:
%sql
-- list all the products that have been sold more than 2 times in total

select od.category as Category_Name, count(od.category) as Category_Count
from order_details od
group by od.category
having count(od.category) > 2
order by od.category asc

In [0]:
%sql
-- list customers and their total quantity of the products ordered
-- using join and cte

with customer_orders as 
(
select c.customer_id, c.first_name, sum(od.quantity) as total_quantity 
from customers c
inner join orders o
  ON c.customer_id = o.customer_id
inner join order_details od
  ON o.order_id = od.order_id
group by c.customer_id, c.first_name
having total_quantity > 3
order by c.customer_id asc
)
select * from customer_orders --where total_quantity > 3

In [0]:
%sql
-- find products with their total sales revenue (price * quantity)
with total_sales as (
select od.category, sum(od.quantity * od.price_each) as total_sales
from order_details od
group by od.category
order by total_sales asc
)
select * from total_sales where total_sales > 2000

**_CASE STATEMENT (SQL- IF/ELSE)_**

In [0]:
%sql
--categorize each order as 'small', 'median' and 'large'
-- using cte and case statement

with order_size_details as 
(
  select o.order_id, o.total_amount,
      case
        when o.total_amount > 2500 then 'large'
        when o.total_amount between 1500 and 2500 then 'median'
        else 'small'
      end as order_size
  from orders o
  order by total_amount asc
)
select * from order_size_details where order_size = 'large'

In [0]:
%sql
-- Label customers as new or returning based on order count
-- using cte and join

with customer_type_details (
  select c.customer_id, count(o.order_id) as total_orders,
      case
          when count(o.order_id) = 1 then 'new'
          when count(o.order_id) > 1 then 'retruning'
          else 'no orders'
      end as customer_type
  from customers c
  left join orders o
  on c.customer_id = o.customer_id
  group by c.customer_id
  order by c.customer_id asc, total_orders asc)

select * from customer_type_details where customer_type = 'new'

**_FUNCTIONS (STRING + DATE)_**

In [0]:
%sql
select upper(first_name) as FIRST_NAME, upper(last_name) as LAST_NAME, concat(first_name, ' ', last_name) from customers  

In [0]:
%sql
-- Extract each month from order_date column and group month and sum total_amount

with order_by_month as (
    select 
        month(order_date) as Month_Order, 
        sum(total_amount) as Total_Amount, 
        count(order_id) as Order_Count
    from orders 
  group by Month_Order
  order by Month_Order asc
)

select * from order_by_month where order_count > 5

In [0]:
%sql
--list each customer's name + their total spend + spending category

with categoty_spend (
select 
    concat_ws(' ', c.first_name, c.last_name) as full_name, 
    od.category, 
    sum(o.total_amount) as total_spend
from customers c
inner join orders o
    on c.customer_id = o.customer_id
inner join order_details od
    on o.order_id = od.order_id
group by c.first_name, c.last_name, od.category
order by full_name)

select * from categoty_spend where total_spend > 1500

In [0]:
%sql
--list each customer's name + their total spend + spending category

with categoty_spend (
select 
    concat_ws(' ', c.first_name, c.last_name) as full_name, 
    c.customer_id, 
    sum(od.quantity * od.price_each) as total_spend
from customers c
inner join orders o
    on c.customer_id = o.customer_id
inner join order_details od
    on o.order_id = od.order_id
group by c.customer_id, c.first_name, c.last_name
order by full_name)

select customer_id, full_name, total_spend,
case
  when total_spend > 20000 then 'Gold'
  when total_spend between 5000 and 20000 then 'Silver'
  else 'Bronze'
end as customer_tier
from categoty_spend;

**_1. String Functions_**

**_- Function	Syntax_**
- LOWER()	LOWER(string)
- UPPER()	UPPER(string)
- LENGTH() / LEN()	LENGTH(string)
- TRIM()	TRIM(string)
- LTRIM()	LTRIM(string)
- RTRIM()	RTRIM(string)
- SUBSTRING()	SUBSTRING(string, start, length)
- REPLACE()	REPLACE(string, search, replace)
- CONCAT()	CONCAT(string1, string2, ...)
- CONCAT_WS()	CONCAT_WS(separator, string1, string2, ...)
- LEFT()	LEFT(string, number_of_chars)
- RIGHT()	RIGHT(string, number_of_chars)

**_2. Date Functions_**

**_- Function	Syntax_**
- CURRENT_DATE()	CURRENT_DATE()
- CURRENT_TIMESTAMP()	CURRENT_TIMESTAMP()
- NOW()	NOW()
- DATE_ADD()	DATE_ADD(date, days)
- DATE_SUB()	DATE_SUB(date, days)
- DATEDIFF()	DATEDIFF(date1, date2)
- YEAR()	YEAR(date)
- MONTH()	MONTH(date)
- DAY()	DAY(date)
- DAYOFWEEK()	DAYOFWEEK(date)
- DATE_FORMAT()	DATE_FORMAT(date, format)

**_3. Numeric Functions_**

**_- Function	Syntax_**
- ABS()	ABS(number)
- ROUND()	ROUND(number, decimals)
- CEIL() / CEILING()	CEIL(number)
- FLOOR()	FLOOR(number)
- POWER()	POWER(number, exponent)
- SQRT()	SQRT(number)
- RAND()	RAND()

**_4. Aggregate Functions_**

**_- Function	Syntax_**
- COUNT()	COUNT(column_or_*)
- SUM()	SUM(column)
- AVG()	AVG(column)
- MIN()	MIN(column)
- MAX()	MAX(column)

**_5. Conditional Functions_**

**_- Function	Syntax_**
- CASE	CASE WHEN condition THEN value ELSE value END
- COALESCE()	COALESCE(expr1, expr2, ...)
- NULLIF()	NULLIF(expr1, expr2)